In [49]:
from keras.applications import MobileNet

# MobileNet was designed to work on 224 x 224 pixel input images sizes
img_rows, img_cols = 224, 224 

# Re-loads the MobileNet model without the top or FC layers
MobileNet = MobileNet(weights = 'imagenet', 
                 include_top = False, 
                 input_shape = (img_rows, img_cols, 3))

# Here we freeze the last 4 layers 
# Layers are set to trainable as True by default
for layer in MobileNet.layers:
    layer.trainable = False
    
# Let's print our layers 
# for (i,layer) in enumerate(MobileNet.layers):
#     print(str(i) + " "+ layer.__class__.__name__, layer.trainable)

In [50]:
def addTopModelMobileNet(bottom_model, num_classes):
    #creates the top or head of the model that will be
    #placed ontop of the bottom layers

    top_model = bottom_model.output
    top_model = GlobalAveragePooling2D()(top_model)
    top_model = Dense(1024,activation='relu')(top_model)
    top_model = Dense(1024,activation='relu')(top_model)
    top_model = Dense(512,activation='relu')(top_model)
    top_model = Dense(num_classes,activation='softmax')(top_model)
    return top_model

In [51]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model

num_classes = 3

FC_Head = addTopModelMobileNet(MobileNet, num_classes)

model = Model(inputs = MobileNet.input, outputs = FC_Head)

print(model.summary())

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128       
_________________________________________________________________
conv_dw_1_relu (ReLU)        (None, 112, 112, 32)      0   

In [52]:
from keras.preprocessing.image import ImageDataGenerator

train_data_dir = '../input/waste-data/DATASET/TRAIN'
validation_data_dir = '../input/waste-data/DATASET/TEST'

# Let's use some data augmentaiton 
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=45,
      width_shift_range=0.3,
      height_shift_range=0.3,
      horizontal_flip=True,
      fill_mode='nearest')
 
validation_datagen = ImageDataGenerator(rescale=1./255)
 
# set our batch size (typically on most mid tier systems we'll use 16-32)
batch_size = 32
 
train_generator = train_datagen.flow_from_directory(train_data_dir,target_size=(img_rows, img_cols),batch_size=batch_size,class_mode='categorical')
 
validation_generator = validation_datagen.flow_from_directory(validation_data_dir,target_size=(img_rows, img_cols),batch_size=batch_size,class_mode='categorical')

Found 24591 images belonging to 3 classes.
Found 2559 images belonging to 3 classes.


In [53]:
print(train_generator.class_indices)

{'O': 0, 'R': 1, 'T': 2}


In [54]:
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping

                     
checkpoint = ModelCheckpoint("./trash_mobilenet.h5",
                             monitor="val_accuracy",
                             mode="max",
                             save_best_only = True,
                             verbose=1)

# earlystop = EarlyStopping(monitor = 'val_accuracy', 
#                           min_delta = 0, 
#                           patience = 10,
#                           verbose = 1,
#                           restore_best_weights = True)

# we put our call backs into a callback list
callbacks = [checkpoint]

# We use a very small learning rate 
model.compile(loss = 'categorical_crossentropy',
              optimizer = RMSprop(lr = 0.0001),
              metrics = ['accuracy'])

# Enter the number of training and validation samples here
nb_train_samples = 24591
nb_validation_samples = 2559

# We only train 5 EPOCHS 
epochs = 50
batch_size = 256

history = model.fit_generator(
    train_generator,
    steps_per_epoch = nb_train_samples // batch_size,
    epochs = epochs,
    callbacks = callbacks,
    validation_data = validation_generator,
    validation_steps = nb_validation_samples // batch_size)

Epoch 1/50
96/96 [==============================] - 39s 388ms/step - loss: 0.4815 - accuracy: 0.7986 - val_loss: 0.2278 - val_accuracy: 0.9201

Epoch 00001: val_accuracy improved from -inf to 0.92014, saving model to ./trash_mobilenet.h5
Epoch 2/50
96/96 [==============================] - 37s 382ms/step - loss: 0.2522 - accuracy: 0.9065 - val_loss: 0.3280 - val_accuracy: 0.8750

Epoch 00002: val_accuracy did not improve from 0.92014
Epoch 3/50
96/96 [==============================] - 37s 382ms/step - loss: 0.2474 - accuracy: 0.9131 - val_loss: 0.2412 - val_accuracy: 0.9306

Epoch 00003: val_accuracy improved from 0.92014 to 0.93056, saving model to ./trash_mobilenet.h5
Epoch 4/50
96/96 [==============================] - 37s 381ms/step - loss: 0.2512 - accuracy: 0.9091 - val_loss: 0.3263 - val_accuracy: 0.8715

Epoch 00004: val_accuracy did not improve from 0.93056
Epoch 5/50
96/96 [==============================] - 37s 390ms/step - loss: 0.1998 - accuracy: 0.9305 - val_loss: 0.3149 - v

96/96 [==============================] - 37s 383ms/step - loss: 0.1295 - accuracy: 0.9567 - val_loss: 0.3813 - val_accuracy: 0.9028

Epoch 00041: val_accuracy did not improve from 0.94444
Epoch 42/50
96/96 [==============================] - 36s 380ms/step - loss: 0.1456 - accuracy: 0.9457 - val_loss: 0.6825 - val_accuracy: 0.8368

Epoch 00042: val_accuracy did not improve from 0.94444
Epoch 43/50
96/96 [==============================] - 37s 380ms/step - loss: 0.1499 - accuracy: 0.9459 - val_loss: 0.7815 - val_accuracy: 0.8993

Epoch 00043: val_accuracy did not improve from 0.94444
Epoch 44/50
96/96 [==============================] - 37s 384ms/step - loss: 0.1636 - accuracy: 0.9408 - val_loss: 0.4529 - val_accuracy: 0.9062

Epoch 00044: val_accuracy did not improve from 0.94444
Epoch 45/50
96/96 [==============================] - 37s 380ms/step - loss: 0.1259 - accuracy: 0.9562 - val_loss: 0.2370 - val_accuracy: 0.9201

Epoch 00045: val_accuracy did not improve from 0.94444
Epoch 46/50


In [55]:
model.save_weights("./trash_mobilenet_v1.h5")

In [57]:
model.save("./trash_mobilenet_v1.h5")

In [58]:
from keras.models import load_model
import tensorflow as tf
classifier = load_model('./trash_mobilenet.h5')
classifier2 = load_model('./trash_mobilenet_v1.h5')

In [59]:
import os
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join
from keras.preprocessing import image
from IPython.display import Image

In [73]:
def predict_external_image(image_path):
    trash_dict = {"[0]": "organic/compost", 
                          "[1]": "Recyclable",
                          "[2]": "Trash"}

    #cv2.destroyAllWindows()
    #subject = "./trashData/validation//n1/cardboard301.jpg"
    subject = image_path
    input_im = cv2.imread(subject)  
    input_original = input_im.copy()
    input_original = cv2.resize(input_original, None, fx=0.5, fy=0.5, interpolation = cv2.INTER_LINEAR)
    input_im = cv2.resize(input_im, (224, 224), interpolation = cv2.INTER_LINEAR)
    input_im = input_im / 255.
    input_im = input_im.reshape(1,224,224,3) 

    # Get Prediction
    res = np.argmax(classifier.predict(input_im, 1, verbose = 0), axis=1)
    print(trash_dict[str(res)])
    print(res)
    print(classifier.predict(input_im, 1, verbose = 0))
    Image(filename = subject)

In [67]:
predict_external_image("../input/test234/image8.jpg")

Recyclable
[1]
[[1.0350773e-01 8.9616263e-01 3.2966139e-04]]


In [68]:
predict_external_image("../input/test234/image9.jpg")

Recyclable
[1]
[[5.0096244e-02 9.4949520e-01 4.0852328e-04]]


In [82]:
predict_external_image("../input/apple-test/image13.jpg")

organic/compost
[0]
[[9.7321790e-01 2.6778076e-02 4.0665723e-06]]


In [81]:
predict_external_image("../input/orange/image14.jpg")

organic/compost
[0]
[[8.9772063e-01 1.0223082e-01 4.8470807e-05]]
